In [27]:
import requests
import pandas as pd
import json
from datetime import datetime
import time
import os
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor

load_dotenv()

True

In [28]:
with open('openapi.json', 'rb') as f:
    cnf = json.load(f)

In [29]:
class ApiClient:
    def __init__(self, token: str, cnf: dict):
        self.token = token
        self.cnf = cnf
        self.url_stat = 'https://statistics-api.wildberries.ru'
        self.session = requests.session()
        self.session.headers.update({'Authorization': self.token})
    
    def __request_get(self, path: str, params: dict):
        while True:
            try:
                r = self.session.get(url = self.url_stat + path, params = params, timeout = 60)
                if r.status_code != 200:
                    if r.json()['errors'] == ['(api-new) too many requests']:
                        time.sleep(60)
                    else:
                        break
                else:
                    break
            except:
                time.sleep(10)
        if r.status_code != 200:
            raise Exception(r.json()['errors'])
        r = r.json()
        r = pd.DataFrame(r)
        
        return r
    
    def __rename_columns(self, data: pd.DataFrame, path: str):
        cols = self.cnf['paths'][path]['get']['responses']['200']['content']\
            ['application/json']['schema']['items']['$ref']
        x = self.cnf
        for c in cols.split('/')[1:]:
            x = x[c]
        cols = x['properties']
        for i, c in enumerate(data.columns):
            try:
                data = data.rename(columns = {c: cols[c]['description'].split('.')[0]})
            except:
                pass
        
        return data
    
    def incomes(self,
                dateFrom: str = '2019-06-20',
                is_russian: bool = True):
        params = locals()
        params.pop('self', 'is_russian')
        path = '/api/v1/supplier/incomes'
        data = self.__request_get(path = path, params = params)
        if is_russian:
            data = self.__rename_columns(data = data, path = path)
        return data
    
    def stocks(self,
               dateFrom: str = '2019-06-20',
               is_russian: bool = True):
        params = locals()
        params.pop('self', 'is_russian')
        path = '/api/v1/supplier/stocks'
        data = self.__request_get(path = path, params = params)
        if is_russian:
            data = self.__rename_columns(data = data, path = path)
        return data
    
    def orders(self,
               dateFrom: str = '2019-06-20',
               is_russian: bool = True):
        params = locals()
        params.pop('self', 'is_russian')
        path = '/api/v1/supplier/orders'
        data = self.__request_get(path = path, params = params)
        if is_russian:
            data = self.__rename_columns(data = data, path = path)
        return data
    
    def sales(self,
              dateFrom: str = '2019-06-20',
              is_russian: bool = True):
        params = locals()
        params.pop('self', 'is_russian')
        path = '/api/v1/supplier/sales'
        data = self.__request_get(path = path, params = params)
        if is_russian:
            data = self.__rename_columns(data = data, path = path)
        return data
    
    def reportDetailByPeriod(self,
                             dateFrom: str = '2019-06-20',
                             dateTo: str = str(datetime.now().date()),
                             is_russian: bool = True):
        params = locals()
        params.pop('self', 'is_russian')
        params['rrdid'] = 0
        path = '/api/v1/supplier/reportDetailByPeriod'
        data = pd.DataFrame()
        while True:
            _data = self.__request_get(path = path, params = params)
            if _data.shape[0] > 0:
                params['rrdid'] = _data['rrd_id'].iloc[-1]
                if is_russian:
                    _data = self.__rename_columns(_data, path = path)
                data = pd.concat([data, _data])
            else:
                break
        return data

In [30]:
token = os.environ['WB_token_stat']

client = ApiClient(token = token, cnf = cnf)

reports = [('incomes', 'Поставки'),
           ('stocks', 'Склад'),
           ('orders', 'Заказы'),
           ('sales', 'Продажи'),
           ('reportDetailByPeriod', 'Отчет о продажах по реализации')]

def save_data(method: str, report: str):
    data = client.__getattribute__(method).__call__()
    data.to_excel(f'{report}.xlsx', index = False)
    return

with ThreadPoolExecutor() as executor:
    futures = []
    for method, report in reports:
        futures.append(
            executor.submit(save_data, method, report)
            )

# for method, report in reports:
#     data = client.__getattribute__(method).__call__()
#     data.to_excel(f'{report}.xlsx', index = False)

client.session.close()